In [9]:
#
# Please execute on QITE_experiments branch from GitHub
# 

from qrisp import QuantumVariable, h, x, prepare, conjugate
from qrisp.vqe import VQEProblem
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function, heisenberg_problem, create_heisenberg_hamiltonian
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
from compile_QITE import compile_QITE

import pickle
from time import time
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

In [13]:
L = 50
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

In [14]:
from qrisp.operators import X, Y, Z

J = 1
B = 0
H = create_heisenberg_hamiltonian(G, J, B)
print(H)

# Define scaling factor
F = 10

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps
def exp_H(qv, t):
    H.trotterization(method='commuting')(qv,t/F,5)

# Hamiltonian simulation via second order Suzuki-Trotter formula with 2 steps
def exp_H_2(qv, t):
    H.trotterization(order=2,method='commuting')(qv,t/F,2)

X(0)*X(1) + X(1)*X(2) + X(10)*X(11) + X(11)*X(12) + X(12)*X(13) + X(13)*X(14) + X(14)*X(15) + X(15)*X(16) + X(16)*X(17) + X(17)*X(18) + X(18)*X(19) + X(19)*X(20) + X(2)*X(3) + X(20)*X(21) + X(21)*X(22) + X(22)*X(23) + X(23)*X(24) + X(24)*X(25) + X(25)*X(26) + X(26)*X(27) + X(27)*X(28) + X(28)*X(29) + X(29)*X(30) + X(3)*X(4) + X(30)*X(31) + X(31)*X(32) + X(32)*X(33) + X(33)*X(34) + X(34)*X(35) + X(35)*X(36) + X(36)*X(37) + X(37)*X(38) + X(38)*X(39) + X(39)*X(40) + X(4)*X(5) + X(40)*X(41) + X(41)*X(42) + X(42)*X(43) + X(43)*X(44) + X(44)*X(45) + X(45)*X(46) + X(46)*X(47) + X(47)*X(48) + X(48)*X(49) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + X(9)*X(10) + Y(0)*Y(1) + Y(1)*Y(2) + Y(10)*Y(11) + Y(11)*Y(12) + Y(12)*Y(13) + Y(13)*Y(14) + Y(14)*Y(15) + Y(15)*Y(16) + Y(16)*Y(17) + Y(17)*Y(18) + Y(18)*Y(19) + Y(19)*Y(20) + Y(2)*Y(3) + Y(20)*Y(21) + Y(21)*Y(22) + Y(22)*Y(23) + Y(23)*Y(24) + Y(24)*Y(25) + Y(25)*Y(26) + Y(26)*Y(27) + Y(27)*Y(28) + Y(28)*Y(29) + Y(29)*Y(30) + Y(3)*Y(4) + Y(30)

## QITE with singlet, plus, vqe warm start

In [15]:
# Tensor product of singlet states
M = nx.maximal_matching(G)
U_singlet = create_heisenberg_init_function(M)

# HVA
H0 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in M)
H1 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in set(G.edges())-M)

def ansatz(qv,theta):
    # Rescaling t->1/4 equivalent to choosing rescaled inital parameters for VQE
    H1.trotterization(method='commuting')(qv,t=theta[1]/4)
    H0.trotterization(method='commuting')(qv,t=theta[0]/4)

def U_vqe(qv):
    return ansatz(qv, [1,1])

# Calculate energy for HVA
def state_prep():
    qv = QuantumVariable(L)
    U_vqe(qv)
    return qv

#print(H.expectation_value(state_prep)())

results = []

results.append(compile_QITE(H, U_singlet, exp_H_2, [1,1,1,1,1], 3, method='GC'))
results.append(compile_QITE(H, U_vqe, exp_H_2, [1,1,1,1,1], 3, method='GC'))

In [10]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
#save_data(results,'data/circuit_benchmarks_2nd_L='+str(L)+'_'+date_time_string)

In [5]:
results

[{'circuit_data': [{0: {'u3': 75, 'cz': 25},
    1: {'u3': 2527, 'cz': 1551},
    2: {'u3': 9104, 'cz': 5695},
    3: {'u3': 28835, 'cz': 18127}},
   {0: 50, 1: 52, 2: 52, 3: 52},
   {0: 3, 1: 519, 2: 2031, 3: 6567}]},
 {'circuit_data': [{0: {'u3': 343, 'cz': 196},
    1: {'u3': 3378, 'cz': 2064},
    2: {'u3': 11707, 'cz': 7234},
    3: {'u3': 36694, 'cz': 22744}},
   {0: 50, 1: 52, 2: 52, 3: 52},
   {0: 17, 1: 561, 2: 2157, 3: 6945}]}]